# Import libraries

In [ ]:
!pip install rdkit

In [ ]:
import rdkit, rdkit.Chem, rdkit.Chem.Draw
from rdkit.Chem import Descriptors
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy import stats

# Read the database

In [ ]:
!wget https://raw.githubusercontent.com/CompMedChemLab/medchem-ml-simulations/main/herg_descriptors.csv.zip
!unzip herg_descriptors.csv.zip

In [ ]:
herg_sm=pd.read_csv("/content/herg_descriptors.csv")

In [ ]:
herg_sm

In [ ]:
molecules=[rdkit.Chem.MolFromSmiles(smi) for smi in herg_sm['SMILES']]

In [ ]:
molecules[10000]

In [ ]:
import random

In [ ]:
random.seed(0)
subset = random.sample(molecules,10)

In [ ]:
img=rdkit.Chem.Draw.MolsToGridImage(subset,molsPerRow=5,subImgSize=(300,300))
img

In [ ]:
print(np.min(herg_sm["pIC50"]))

In [ ]:
molecules[np.argmin(herg_sm["pIC50"])]

In [ ]:
print(np.max(herg_sm["pIC50"]))

In [ ]:
molecules[np.argmax(herg_sm["pIC50"])]

In [ ]:
molecules[np.argmin((herg_sm["pIC50"].mean()-herg_sm["pIC50"])**2)]

In [ ]:
valid_mols= [x for x in molecules if x != None]

In [ ]:
len (molecules),len(valid_mols)

In [ ]:
y=herg_sm['pIC50']
features=herg_sm[herg_sm.columns[0:-2]]

In [ ]:
features

# Scaling of variables

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
features_scaled=pd.DataFrame(MinMaxScaler().fit_transform(features), columns=features.columns)

# Multiple Linear Regression

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(features,y,test_size=0.25)

In [ ]:
reg = LinearRegression().fit(X_train,Y_train)
Y_test_predict=reg.predict(X_test)
Y_train_predict=reg.predict(X_train)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k')
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1)

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
stats.pearsonr(Y_train,Y_train_predict)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k')
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1)
plt.plot(Y_test,Y_test_predict,'.')

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
stats.pearsonr(Y_test,Y_test_predict)

In [ ]:
plt.figure(figsize=(6,5))
plt.hist(y, bins=100,edgecolor='k',range=(0,12))
plt.xlabel('Actual pIC50')
plt.ylabel('Probability')

## Balancing

In [ ]:
n_bins=100
bins=np.linspace(0,12,n_bins+1)

In [ ]:
index_bin=[np.where((y>=bins[i]) & (y<bins[i+1]))[0] for i in range(n_bins)]

In [ ]:
selected=[np.random.choice(x,size=min(len(x),np.random.randint(40,50)), replace=False) for x in index_bin if len(x)>0]

In [ ]:
balanced_y=np.concatenate([y[x] for x in selected])

In [ ]:
balanced_features=np.concatenate([np.array(features_scaled)[x] for x in selected])

In [ ]:
balanced_molecules=[molecules[x] for x in np.concatenate(selected)]

In [ ]:
plt.figure(figsize=(6,5))
plt.hist(balanced_y,bins=100,edgecolor='k',range=(0,12))
plt.xlabel('Actual pIC50')
plt.ylabel('Probability')

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(balanced_features,balanced_y,test_size=0.25)

In [ ]:
reg = LinearRegression().fit(X_train,Y_train)
Y_test_predict=reg.predict(X_test)
Y_train_predict=reg.predict(X_train)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k')
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1)

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
stats.pearsonr(Y_train,Y_train_predict)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k')
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1)
plt.plot(Y_test,Y_test_predict,'.')

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
stats.pearsonr(Y_test,Y_test_predict)

# Principal Component Regression

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA()

In [ ]:
new_values=pca.fit_transform(balanced_features)

In [ ]:
plt.bar(np.linspace(0,770,771),pca.explained_variance_ratio_)
plt.xlim([-0.5,10.5])
plt.xlabel("Eigenvector #")
plt.ylabel("Explained Variance Ratio")

In [ ]:
threshold=0.9

In [ ]:
np.min(np.where(np.cumsum(pca.explained_variance_ratio_)>threshold)[0])

In [ ]:
reg = LinearRegression().fit(X_train[:,:np.min(np.where(np.cumsum(pca.explained_variance_ratio_)>threshold)[0])],Y_train)
Y_test_predict=reg.predict(X_test[:,:np.min(np.where(np.cumsum(pca.explained_variance_ratio_)>threshold)[0])])
Y_train_predict=reg.predict(X_train[:,:np.min(np.where(np.cumsum(pca.explained_variance_ratio_)>threshold)[0])])

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k')
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1)

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
stats.pearsonr(Y_train,Y_train_predict)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k')
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1)
plt.plot(Y_test,Y_test_predict,'.')

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
stats.pearsonr(Y_test,Y_test_predict)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
errors=np.empty((len(balanced_features[0,:np.min(np.where(np.cumsum(pca.explained_variance_ratio_)>0.999)[0])]),2))
for i in range (len(balanced_features[0,:np.min(np.where(np.cumsum(pca.explained_variance_ratio_)>0.999)[0])])):
  X_train_touse=X_train[:,:i+1]
  X_test_touse=X_test[:,:i+1]
  reg = LinearRegression().fit(X_train_touse,Y_train)
  Y_test_predict=reg.predict(X_test_touse)
  Y_train_predict=reg.predict(X_train_touse)
  errors[i,0]=mean_squared_error(Y_train,Y_train_predict)
  errors[i,1]=mean_squared_error(Y_test,Y_test_predict)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(errors[1:,0][::1],label='Train')
plt.plot(errors[1:,1][::1],label='Test')
plt.ylim(0,3)

plt.xlabel('# Components')
plt.ylabel('Error')

plt.grid()
plt.legend()

In [ ]:
np.argmin(errors[1:,1])

In [ ]:
reg = LinearRegression().fit(X_train[:,:np.argmin(errors[1:,1])],Y_train)
Y_test_predict=reg.predict(X_test[:,:np.argmin(errors[1:,1])])
Y_train_predict=reg.predict(X_train[:,:np.argmin(errors[1:,1])])

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k')
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1)
plt.plot(Y_test,Y_test_predict,'.')

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
stats.pearsonr(Y_train,Y_train_predict)

In [ ]:
stats.pearsonr(Y_test,Y_test_predict)

## Best predictions and Outliers

In [ ]:
data=Y_test/Y_test_predict

In [ ]:
data.mean()

In [ ]:
data.std()

In [ ]:
best=[]
index_best=[]
for i in np.sort(np.abs(data-1))[:10]:
  index_best.append(np.where(np.abs(data-1)==i)[0][0])
  best.append(balanced_molecules[np.where(np.abs(data-1)==i)[0][0]])

In [ ]:
index_best

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k',zorder=0)
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1,zorder=0)
plt.plot(Y_test,Y_test_predict,'.',zorder=0)
plt.scatter(Y_test[index_best],Y_test_predict[index_best], edgecolor="black",zorder=1)

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
img=rdkit.Chem.Draw.MolsToGridImage(best,molsPerRow=4,subImgSize=(300,300))
img

In [ ]:
index_outliers=np.where(np.logical_or(data>(data.mean()+data.std()*2),(data<(data.mean()-data.std()*2))))

In [ ]:
outliers=[]
for i in index_outliers[0]:
  outliers.append(balanced_molecules[i])

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k',zorder=0)
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1,zorder=0)
plt.plot(Y_test,Y_test_predict,'.',zorder=0)
plt.scatter(Y_test[index_outliers],Y_test_predict[index_outliers], edgecolor="black",c='Tab:Red',zorder=1)

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
img=rdkit.Chem.Draw.MolsToGridImage(outliers,molsPerRow=4,subImgSize=(300,300))
img

# Regularization Methods

## 1. Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
lambda_values=[1e-9,1e-8,1e-7,1e-6,1e-5,1e-4,2.5e-4,5e-4,7.5e-4,1e-3,2.5e-3,5e-3,7.5e-3,1e-2,2.5e-2,5e-2,7.5e-2,0.1,0.25,0.5,0.75,1,10,1e2,1e3,1e4,1e5,1e6,1e7,1e8,1e9]

In [ ]:
errors_ridge=np.empty((len(lambda_values),2))
for n,i in enumerate(lambda_values):
  reg=Ridge(i).fit(X_train,Y_train)
  Y_test_predict=reg.predict(X_test)
  Y_train_predict=reg.predict(X_train)
  errors_ridge[n,0]=mean_squared_error(Y_train,Y_train_predict)
  errors_ridge[n,1]=mean_squared_error(Y_test,Y_test_predict)

In [ ]:
print(errors_ridge[:,1])

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(errors_ridge[:,0],label='Train')
plt.plot(errors_ridge[:,1],label='Test')

plt.xticks(np.arange(0,len(lambda_values)),lambda_values, rotation=45, fontsize=8)

plt.ylim(0,3)
plt.xlabel('Regularization Strength')
plt.ylabel('Error')

plt.grid()
plt.legend()

In [ ]:
errors_ridge[:,1]

In [ ]:
np.argmin(errors_ridge[:,1])

In [ ]:
lambda_values[np.argmin(errors_ridge[:,1])]

In [ ]:
reg=Ridge(lambda_values[np.argmin(errors_ridge[:,1])]).fit(X_train,Y_train)
Y_test_predict=reg.predict(X_test)
Y_train_predict=reg.predict(X_train)
errors_ridge[n,0]=mean_squared_error(Y_train,Y_train_predict)
errors_ridge[n,1]=mean_squared_error(Y_test,Y_test_predict)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k')
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1)
plt.plot(Y_test,Y_test_predict,'.')

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
stats.pearsonr(Y_train,Y_train_predict)

In [ ]:
stats.pearsonr(Y_test,Y_test_predict)

In [ ]:
reg.coef_

In [ ]:
np.argsort(reg.coef_)[-10:]

In [ ]:
herg_sm.columns[np.argsort(reg.coef_)[-10:]]

## Check the [Feature Set](https://mordred-descriptor.github.io/documentation/master/descriptors.html)

## 2. Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lambda_values=[1e-9,1e-8,1e-7,1e-6,1e-5,1e-4,2.5e-4,5e-4,7.5e-4,1e-3,2.5e-3,5e-3,7.5e-3,1e-2,2.5e-2,5e-2,7.5e-2,0.1,0.25,0.5,0.75,1,10,1e2,1e3,1e4,1e5,1e6,1e7,1e8,1e9]

In [ ]:
errors_lasso=np.empty((len(lambda_values),2))
for n,i in enumerate(lambda_values):
  print(n,i)
  reg=Lasso(i,max_iter=int(2e4)).fit(X_train,Y_train)
  Y_test_predict=reg.predict(X_test)
  Y_train_predict=reg.predict(X_train)
  errors_lasso[n,0]=mean_squared_error(Y_train,Y_train_predict)
  errors_lasso[n,1]=mean_squared_error(Y_test,Y_test_predict)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(errors_lasso[:,0],label='Train')
plt.plot(errors_lasso[:,1],label='Test')

plt.xticks(np.arange(0,len(lambda_values)),lambda_values, rotation=45, fontsize=8)

plt.ylim(0,3)
plt.xlabel('Regularization Strength')
plt.ylabel('Error')

plt.grid()
plt.legend()

In [ ]:
lambda_values[np.argmin(errors_lasso[:,1])]

In [ ]:
reg=Lasso(lambda_values[np.argmin(errors_lasso[:,1])],max_iter=int(2e4)).fit(X_train,Y_train)
Y_test_predict=reg.predict(X_test)
Y_train_predict=reg.predict(X_train)
errors_lasso[n,0]=mean_squared_error(Y_train,Y_train_predict)
errors_lasso[n,1]=mean_squared_error(Y_test,Y_test_predict)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

plt.plot(np.linspace(0,12),np.linspace(0,12),c='k')
plt.plot(Y_train,Y_train_predict,'.', alpha=0.1)
plt.plot(Y_test,Y_test_predict,'.')

plt.xlim(0,12)
plt.ylim(0,12)
plt.xlabel('Actual pIC50')
plt.ylabel('Predicted pIC50')

ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
stats.pearsonr(Y_train,Y_train_predict)

In [ ]:
stats.pearsonr(Y_test,Y_test_predict)

In [ ]:
reg.coef_

In [ ]:
np.argsort(reg.coef_)[-10:]

In [ ]:
herg_sm.columns[np.argsort(reg.coef_)[-10:]]

## Check the [Feature Set](https://mordred-descriptor.github.io/documentation/master/descriptors.html)